In [ ]:
import os
import time
import openai
import requests

import pandas as pd
from tqdm import tqdm

# Create Dataset

In [ ]:
df = pd.read_csv('./bert-multilingual/toxic/jigsaw-unintended-bias-train_fr_clean.csv').sample(frac=1,random_state=42)

# Ignore severe_toxic as they are very few and ve can use toxic confidence as severity.
labels = ['toxic','obscene', 'threat', 'insult', 'identity_attack']
for label in labels:
    df[label] = (df[label] > 0.5).astype(int)

In [ ]:
df1 = pd.concat((
  df[(df.toxic == 1) & (df.threat==0)].head(300), # almost all the threat examples contains toxic as second label, seperate them
  df[df.threat==1].head(100),
  df[((df.toxic == 0) & (df.severe_toxicity == 0) & (df.obscene == 1) & (df.threat == 0) & (df.insult == 0) & (df.identity_attack == 0))].head(100), # very few obscene in toxics, increase population
  df[((df.toxic == 0) & (df.severe_toxicity == 0) & (df.obscene == 0) & (df.threat == 0) & (df.insult == 1) & (df.identity_attack == 0))].head(50), # add some insult without toxic
  df[((df.toxic == 0) & (df.severe_toxicity == 0) & (df.obscene == 0) & (df.threat == 0) & (df.insult == 0) & (df.identity_attack == 1))].head(100), # add some identity_attack without toxic
))

# add complete neutral samples
df1 = pd.concat((
  df1,
  df[((df.toxic == 0) & (df.severe_toxicity == 0) & (df.obscene == 0) & (df.threat == 0) & (df.insult == 0) & (df.identity_attack == 0))].head(1000-len(df1))
))

df1 = df1.rename(columns={'comment_text':'text'})
df1 = df1[['text']+labels]
len(df1)

In [ ]:
for label in labels:
    print(label, df1[label].value_counts().to_dict())

In [ ]:
df1.to_csv('./jigsaw_test_data.csv',index=False)

In [ ]:
#TODO: delete
df1 = pd.read_csv('prediction_results.csv')
len(df1)

# OpenAI Multi-Label Evaluation

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
def classify_text(input_text, model="text-davinci-003", retry=0):    
    prompt = f"""The following is a list of text categories:
    1. toxic
    2. obscene
    3. threat
    4. insult
    5. identity_attack
    Classify the following French text as one or multiple of the above categories (Write numbers, no text):
    "{input_text}"
    """
    
    response = openai.Completion.create(
        model=model,
        prompt=prompt,
        temperature=0,
        max_tokens=100,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    label = response["choices"][0].text.strip()    
    return label

def classify_text_binary(input_text, model="text-davinci-003", retry=0):    
    prompt = f"""Is the following French text is toxic? (just say yes/no):{input_text} """

    response = openai.Completion.create(
        model=model,
        prompt=prompt,
        temperature=0,
        max_tokens=100,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    label = response["choices"][0].text.strip()    
    return label.strip()

def predict_with_retry(text,binary=False, retry=0):
    try:
        if binary:
            return classify_text_binary(text)
        else:
            return classify_text(text)
    except Exception as e:
        if retry < 3:
            print("Error, retrying in 3 sec",e)
            time.sleep(3)
            return predict_with_retry(text, binary=binary, retry = retry+1)
        else:
            print("Reached max retry limit. Return None")
            return None          

In [ ]:
predictions = []
for i,row in tqdm(df1.iterrows(),total=len(df1)):    
    prediction = predict_with_retry(row.text)
    predictions.append(prediction)
    time.sleep(1)
    if len(predictions) % 40 == 0:
        time.sleep(60)

df1['pred_openai'] = predictions        

In [ ]:
predictions = []
for i,row in tqdm(df1.iterrows(),total=len(df1)):    
    prediction = predict_with_retry(row.text,binary=True)
    predictions.append(prediction)
    time.sleep(1)
    if len(predictions) % 40 == 0:
        time.sleep(60)

df1['pred_openai_binary'] = predictions        

# Hive

In [ ]:
API_KEY = os.environ.get("HIVE_API_KEY")

def classify_text(input_text):
    response = requests.post(
        url='https://api.thehive.ai/api/v2/task/sync',
        headers={'Authorization': f'Token {API_KEY}'},
        data={'text_data': input_text},
    )
    response_dict = response.json()
    
    res = response_dict['status'][0]
    return res['response']['output'][0]['classes']

def predict_with_retry(text,retry=0):
    try:
        return classify_text(text)
    except Exception as e:
        if retry < 3:
            print("Error, retrying in 3 sec",e)
            time.sleep(3)
            return predict_with_retry(text, retry = retry+1)
        else:
            print("Reached max retry limit. Return None")
            return None    

In [ ]:
predictions = []
for i,row in tqdm(df1.iterrows(),total=len(df1)):    
    prediction = predict_with_retry(row.text)
    predictions.append(prediction)
    time.sleep(3)

    if len(predictions) % 40 == 0:
        time.sleep(120)

In [ ]:
df1['pred_hive'] = predictions

In [ ]:
df1.to_csv('./prediction_results.csv',index=False)